In [1]:
import numpy as np
import pandas as pd
import os
import keras
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000


Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


In [2]:
#keras specific imports 
from keras import Model, Input
from keras.layers import Dense, Flatten,GlobalMaxPool2D,Dropout
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

from keras.applications import VGG16,vgg16
from keras.applications import VGG19,vgg19
from keras.applications import InceptionV3, inception_v3

In [3]:
#load pretrained model weights
VGG16_base = VGG16(weights = 'imagenet', include_top = False, input_shape= (224,224,3))
VGG19_base = VGG19(weights = 'imagenet', include_top = False, input_shape= (224,224,3))
inception_v3_base = InceptionV3(weights = 'imagenet', include_top = False, input_shape= (300,300,3))

In [7]:
NUMBER_CLASSES = 120

In [96]:
def create_train_validation(processing_func,image_dim):
    data_gen = ImageDataGenerator(shear_range= 0.3,
                                      horizontal_flip= True,
                                      width_shift_range=0.3,
                                      height_shift_range = 0.3,
                                      preprocessing_function= (lambda x: processing_func(x)) )                                          

    train_generator = data_gen.flow_from_directory('./train',
                                                       batch_size = 24,
                                                   target_size= (image_dim,image_dim),
                                                   class_mode= 'categorical')


#creates validation set
    X_val_path, y_val = [], []
    for folder in os.listdir('validation'):
        for path in os.listdir('validation/'+folder):
            y_val.append(folder)
            X_val_path.append('validation/'+folder+'/'+path)

    def prep_from_path(path):
        img = image.load_img(path, target_size = (image_dim,image_dim))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis = 0)
        x = processing_func(x)
        return x.reshape(image_dim,image_dim,3)


    X_val=[]
    for path in X_val_path:
        X_val.append(prep_from_path(path))

    X_val = np.array(X_val)

    class_map = train_generator.class_indices

    y_val = [class_map[y] for y in y_val]

    y_val = to_categorical(y_val)
    return train_generator, X_val, y_val

In [8]:
def create_model(base_layers,layer1_units,layer2_units):
    for layer in base_layers.layers:
        layer.trainable = False
    X = base_layers.output
    X = GlobalMaxPool2D()(X)
    X = Dense(layer1_units, activation = 'relu', name = '1')(X)
    X = Dense(layer2_units, activation = 'relu',name = '2')(X)
    predictions = Dense(NUMBER_CLASSES , activation = 'softmax',name = '3')(X)
    model = Model(inputs = base_layers.inputs, outputs = predictions)
    return model
    

In [91]:
def create_model_one_layer(base_layers,layer1_units):
    for layer in base_layers.layers:
        layer.trainable = False
    X = base_layers.output
    X = GlobalMaxPool2D()(X)
    X = Dense(layer1_units, activation = 'relu', name = '1')(X)
    predictions = Dense(NUMBER_CLASSES , activation = 'softmax',name = '2')(X)
    model = Model(inputs = base_layers.inputs, outputs = predictions)
    return model

In [65]:
def get_top_X_accuracy(actual,predictions,X= 5):
    actual = np.argmax(actual,axis = 1).reshape(-1,1)
    top_X = np.argsort(predictions, axis = 1)[:,-X:]
    acc = np.any( top_X == actual, axis  = 1)
    return np.mean(acc)

In [73]:
early_stopping = EarlyStopping(monitor = 'val_loss',
                               min_delta = .01, 
                               patience = 3,
                               verbose = 1,
                               mode = 'min')


In [ ]:
image_dim = 300
inception_v3_base = InceptionV3(weights = 'imagenet', include_top = False, input_shape= (image_dim,image_dim,3))
model1 = create_model(inception_v3_base,500,200)

model1.compile(optimizer='rmsprop',loss = 'categorical_crossentropy',metrics = ['accuracy'])


train_generator, X_val, y_val = create_train_validation(inception_v3.preprocess_input,image_dim)
model1.fit_generator(generator = train_generator, 
                        epochs=40,
                            steps_per_epoch= 50, #there are 24 images per step so this is 1200 per epoch
                            callbacks = [early_stopping],
                            validation_data = (X_val,y_val),
                            workers=12,
                            use_multiprocessing=True,
                            verbose = 1) 


Found 9254 images belonging to 120 classes.
Epoch 1/40
50/50 [==============================] - 54s 1s/step - loss: 10.6993 - acc: 0.0117 - val_loss: 4.7131 - val_acc: 0.0341
Epoch 2/40
45/50 [==========================>...] - ETA: 1s - loss: 4.8555 - acc: 0.0213

In [88]:
preds1 = model1.predict(X_val)

In [89]:
get_top_X_accuracy(y_val,preds1,5)

0.96590909090909094

In [90]:
get_top_X_accuracy(y_val,preds1,10)

0.98553719008264462

In [64]:
for layer in model1.layers:
    print(layer, layer.trainable, layer.output_shape)

<keras.engine.topology.InputLayer object at 0x7fe0af49a0f0> False (None, 224, 224, 3)
<keras.layers.convolutional.Conv2D object at 0x7fe0aedb3c50> False (None, 111, 111, 32)
<keras.layers.normalization.BatchNormalization object at 0x7fe0aec99588> False (None, 111, 111, 32)
<keras.layers.core.Activation object at 0x7fe0aedb3d30> False (None, 111, 111, 32)
<keras.layers.convolutional.Conv2D object at 0x7fe0aeb97048> False (None, 109, 109, 32)
<keras.layers.normalization.BatchNormalization object at 0x7fe0aec0a780> False (None, 109, 109, 32)
<keras.layers.core.Activation object at 0x7fe0aeba9320> False (None, 109, 109, 32)
<keras.layers.convolutional.Conv2D object at 0x7fe0aeb5b048> False (None, 109, 109, 64)
<keras.layers.normalization.BatchNormalization object at 0x7fe0aeb43be0> False (None, 109, 109, 64)
<keras.layers.core.Activation object at 0x7fe0aeae3710> False (None, 109, 109, 64)
<keras.layers.pooling.MaxPooling2D object at 0x7fe0aea80be0> False (None, 54, 54, 64)
<keras.layers.c